In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import numpy as np
import pandas as pd
from __future__ import print_function, division
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import time
import sys

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [58]:
#========================================================================
# First Month別にFitさせるにあたり、データセットの分布を各First Monthに揃える
#========================================================================

#========================================================================
# Args 
# fm
is_viz = False
limit_diff_num = 5
base_fam = '2017-12'
base_year = int(base_fam[:4])
base_month = int(base_fam[-2:])
max_fam = '2017-12' 
min_fam = '2011-11' 
target_max = np.max(df.dropna().index.tolist())
target_min = np.min(df.dropna().index.tolist())

result_id = []
#========================================================================

#========================================================================
# Data Load
base = utils.read_df_pkl('../input/base_first*')
base[target] = base[target].map(lambda x: np.round(x, 1))
#========================================================================
df_list = []

val_cnt = base[target].value_counts()
val_cnt.name = 'all'
df_list.append(val_cnt.copy())

base_1712 = base[base['first_active_month']==base_fam]
val_cnt = base_1712[target].value_counts()
if is_max:
    val_cnt_max = val_cnt.max()
    val_cnt /= val_cnt_max
val_cnt.name = base_fam
df_list.append(val_cnt)
df = pd.concat(df_list, axis=1)

def arange_ratio(df, multi, is_viz=False):
    df[base_fam] *= multi
    df['diff'] = df['all'] - df[base_fam]
    diff_len = len(df[df['diff']<0])
    if is_viz:
        display(df[df['diff']<0])
    if diff_len > limit_diff_num:
        return -1
    return 0
    
df = pd.concat(df_list, axis=1)
cnt_0_fam = df.loc[0.0, :][base_fam]
cnt_0_all = df.loc[0.0, :]['all']
multi = int(cnt_0_all / cnt_0_fam)+1

while True:
    tmp = df.copy().dropna()
    is_minus = arange_ratio(tmp, multi)
    if is_minus:
        multi -= 1
        continue
    break
    
print(f"multi: {multi}")
df[base_fam] *= multi
df_loy = df.dropna()
loy_list = list(df_loy.index)


#========================================================================
# Sampling
#========================================================================
for i in loy_list:
    loy = np.round(i, 1)
    df_id = base[base[target]==loy]
    if len(df_id)==0:
        continue
    sample = df.loc[loy, base_fam]
    if sample==sample:
        before = sample
    else:
        sample = before
    sample = np.int(sample)
    remain = sample
    sampling_id = []
    
    if is_viz:
        print('''
#========================================================================
# Sampling Start!!
''')
    
    for i in range(100):
        
        is_add = True
        if i==0:
            yyyymm = base_fam
            tmp_id = df_id[df_id['first_active_month']==yyyymm]
        else:
            year = base_year
            month = base_month + i
            
            if month>12:
                num_year = month//12
                year = year + num_year
                month = month - 12 * num_year
            elif month<1:
                num_year = month//12
                if num_year==0:
                    year = year - 1
                    month = month + 12
                else:
                    num_year *= -1
                    year = year - num_year
                    month = month + 12*num_year
            if month<10:
                month = f'0{month}'

            yyyymm = f"{year}-{month}"  

            if yyyymm<min_fam or yyyymm>max_fam:
                is_add = False
            else:
                tmp_id = df_id[df_id['first_active_month']==yyyymm]
                if i>0 and yyyymm==base_fam:
                    is_add = False

        #========================================================================
        # Sampling
        if is_add:
            if is_viz:
                print(f'future yyyymm: {yyyymm}')
            id_list = list(tmp_id[key].values)
            if len(id_list)<=remain:
                sampling_id += id_list
            else:
                sampling_id += list(np.random.choice(id_list, remain, replace=False))
                
            if is_viz:
                print(f"sampling_id: {len(sampling_id)} / {sample}")
        #========================================================================
                
        remain = sample - len(sampling_id)
        if remain <= 0:
            break
                
        is_add = True
        if i>0:
            year = base_year
            month = base_month - i
            
            if month>12:
                num_year = month//12
                year = year + num_year
                month = month - 12 * num_year
            elif month<1:
                num_year = month//12
                if num_year==0:
                    year = year - 1
                    month = month + 12
                else:
                    num_year *= -1
                    year = year - num_year
                    month = month + 12*num_year
            if month<10:
                month = f'0{month}'

            yyyymm = f"{year}-{month}"  
    
            if yyyymm<min_fam or yyyymm>max_fam:
                is_add = False
            else:
                tmp_id = df_id[df_id['first_active_month']==yyyymm]
            
            #========================================================================
            # Sampling
            if is_add:
                if is_viz:
                    print(f'past yyyymm: {yyyymm}')
                id_list = list(tmp_id[key].values)
                if len(id_list)<=remain:
                    sampling_id += id_list
                else:
                    sampling_id += list(np.random.choice(id_list, remain, replace=False))
    
                if is_viz:
                    print(f"sampling_id: {len(sampling_id)} / {sample}")
            #========================================================================
            
            remain = sample - len(sampling_id)
            if remain <= 0:
                break

    result_id += sampling_id
    if is_viz:
        print(f"loy:{loy} | {len(sampling_id)}/{sample} | All: {len(result_id)}")
        print('''
# Sampling Complete!!
#========================================================================
''')
print(f"All: {len(result_id)}")
display(base[base[key].isin(result_id)]['first_active_month'].value_counts().head(10))
display(base[base[key].isin(result_id)]['target'].value_counts().head(10))

100%|██████████| 3/3 [00:00<00:00, 79.92it/s]


multi: 5
All: 68457


2017-09    13820
2017-10    13695
2017-08    13080
2017-11    12909
2017-12    10156
2017-07     4540
2017-06      180
2017-05       51
2017-01        6
2017-04        6
2017-03        5
2016-11        2
2013-11        1
2012-08        1
2015-11        1
2016-06        1
2017-02        1
2016-12        1
2015-01        1
Name: first_active_month, dtype: int64

 0.0     3220
 0.1     2910
-0.1     2620
-0.2     2555
 0.3     2525
 0.2     2480
-0.3     2450
-0.4     2315
 0.4     2200
-0.5     2090
 0.5     1985
 0.6     1985
 0.7     1905
-0.6     1890
-0.7     1655
-0.9     1545
 0.9     1470
 0.8     1455
-0.8     1410
 1.0     1335
-1.0     1320
 1.1     1170
-1.1     1125
 1.2     1080
-1.2      990
-1.3      985
-1.4      970
 1.3      920
 1.4      880
 1.5      825
-1.6      710
 1.6      690
-1.5      655
-1.7      615
 1.7      610
-1.8      595
-1.9      535
 1.8      520
 2.1      480
-2.0      455
-2.2      425
 2.0      425
 1.9      390
 2.3      385
-2.1      380
 2.2      380
-2.4      370
 2.6      345
 2.5      315
 2.4      310
-2.3      285
-2.6      280
-2.5      280
-2.9      265
 3.1      235
-2.7      235
 2.7      230
-2.8      225
-3.0      180
 2.9      175
-3.3      175
 2.8      170
 3.3      150
-3.2      150
 3.7      145
-3.5      140
 3.0      140
-3.1      135
 3.2      130
 4.0      115
 3.4      110
-3.9  